##  Dictionaries

In [0]:

DEFAULT_CODES = {
    "API_TYPE": "apv2",
    "AVANTAGES_SOCIAUX": "asv2",
    "FINANCIAL_SERVICES": "sfv2",
    "CAUTIONNEMENT": "ccn"
}

API_TYPE_MAPPING = {
    "particuliers": "apv2",
    "avantages-sociaux": "asv2",
    "cautionnement": "cv2",
    "entreprises": "epv2",
    "services-financiers": "sfv2",
    "entreprises_grandeentreprise": "egv2",
    "entreprises_agricole": "eav2"
}


AVANTAGES_SOCIAUX_SERVICE_MAPPING = {
    "assurance-collective": "asc",
    "regime-retraite": "rer",
    "consultation-rh": "crh"
}


FINANCIAL_SERVICES_MAPPING = {
    "assurance-salaire": "asa",
    "placement": "plc",
    "hypotheque": "hyp",
    "ligne-personnelle": "lip",
    "ligne-commerciale": "lic",
    "assurances-voyages": "asvo"
}


CAUTIONNEMENT_MAPPING = {
    "cautionnement-contrats": "ccn",
    "cautionnement-commerciaux": "cco",
    "assurance-credit-depots": "acd"
}


ENTREPRISE_TYPES = {
    "epv2": "PME",
    "eav2": "Agricole",
    "egv2": "Grande entreprise",
}


# ---------------------------METHODS--------------------------

In [0]:
def mapReferenceTypeToApiType(referenceType) :
  normalizedType = (referenceType or "").lower()
  return API_TYPE_MAPPING.get(normalizedType, DEFAULT_CODES['API_TYPE'])

In [0]:
def normalizeServicesString(services) :
  if services is None or services == 'null':
    return []
  return [s.strip() for s in str(services).replace("{", "").replace("}", "").split(",") if len(s.strip())>0]  

In [0]:
def mapServicesToApiCodes(services, serviceMapping, defaultCode) : # tested good
  cleaned = normalizeServicesString(services)
  if len(cleaned) == 0 :
    return [defaultCode]
  apiCodes = []
  for service in cleaned :
    lower = service.lower()
    for keyword, code in serviceMapping.items() :
      if keyword in lower and code not in apiCodes :
        apiCodes.append(code)
  return apiCodes if len(apiCodes) > 0 else [defaultCode]

In [0]:
def map_avantages_sociaux_to_api_codes(avantagesSociauxServices): 
  return mapServicesToApiCodes(avantagesSociauxServices, AVANTAGES_SOCIAUX_SERVICE_MAPPING, DEFAULT_CODES['AVANTAGES_SOCIAUX'])


def map_financial_services_to_api_codes(financialServices) :
  return mapServicesToApiCodes(financialServices, FINANCIAL_SERVICES_MAPPING, DEFAULT_CODES['FINANCIAL_SERVICES'])


def map_cautionnement_type_to_api_codes(typeCautionnement) :
  return mapServicesToApiCodes(typeCautionnement, CAUTIONNEMENT_MAPPING, DEFAULT_CODES['CAUTIONNEMENT'])


In [0]:
def _get(record, key, default=None):
    if isinstance(record, dict):
        value = record.get(key, default)
    else:
        value = getattr(record, key, default)
    if value in (None, ""):
        return default
    return value

def _extract_base64_payload(attachment_content: str) -> str:
    if not attachment_content:
        return ""
    idx = attachment_content.find("base64,")
    return attachment_content[idx + 7:] if idx != -1 else attachment_content

def _build_files(record) -> list:
    name = _get(record, "attachment_name")
    content = _get(record, "attachment_content")
    if name and content:
        return [{"name": name, "content": _extract_base64_payload(content)}]
    return []

# builders for each payload
def _build_apv2(record): # Gere apv2
    return {"assuranceParticuliersV2": {"codeEPIC": ""}}

def _build_asv2(record):  # Gere asv2
    return {
        "avantageSociauxV2": {
            "nomEntreprise": _get(record, "company_name", "Entreprise"),
            "nombreEmployes": int(str(_get(record, "number_of_employees", "0")) or "0"),
            "typeService": map_avantages_sociaux_to_api_codes(_get(record, "avantages_sociaux_services")),
        }
    }

def _build_cv2(record): # Gere cv2
    return {
        "cautionnementV2": {
            "nomEntreprise": _get(record, "company_name", "Entreprise"),
            "etreContacte" : True,
            "typeService": map_cautionnement_type_to_api_codes(_get(record, "cautionnement_services")),
            
        }
    }

def _build_services_financiers(record): # Gere sfv2
    return {
        "servicesFinanciersV2": {
            "nomEntreprise": _get(record, "company_name", "Entreprise"),
            "nombreEmployes": int(str(_get(record, "number_of_employees", "1")) or "1"),
            "typeService": map_financial_services_to_api_codes(_get(record, "financial_services")),
        }
    }

def _build_entreprise_common(api_type, record):
    #Gere epv2(PME), eav2(Agricole), egv2(Grande entreprise)
    libelle = ENTREPRISE_TYPES[api_type]
    return {
        "assuranceEntrepriseV2": {
            "nomEntreprise": _get(record, "company_name", "Entreprise"),
            "etreContacte": True,
            "typesEntreprisePossibles": [
                {"id": api_type, "libelle": libelle, "actif": True}
            ],
        }
    }

def get_type_specific_data(api_type: str, record):
    
    api_type = (api_type or "").lower()
    fichiers = _build_files(record)

    if api_type in ENTREPRISE_TYPES: # Builder for common enterprises
        result = _build_entreprise_common(api_type, record)
    else: # Builder for the rest of the api types => Le dic
        builders = {
            "apv2": _build_apv2,
            "asv2": _build_asv2,
            "cv2": _build_cv2,
            "sfv2": _build_services_financiers,
        }
        result = builders.get(api_type, _build_apv2)(record)

    if fichiers: # si fichiers existe => ajouter a result
        result["fichiers"] = fichiers
    return result


FYI Record est de type Dict, donc on utilise : df=spark.sql(query)     for row in df : record = row.asDict()  suite de code 

#   ----------------------MAIN_METHOD()-------------------------

In [0]:
def transform_to_api_format(record):  # fonction main 
    api_type = _get(record, "reference_type", "default")
    api_type = mapReferenceTypeToApiType(api_type)
    type_specific_data = get_type_specific_data(api_type, record)

    payload = {
        "id": _get(record, "id", ""),
        "type": api_type,
        "courrielReferenceur":_get(record, "company_email"),
        "nomReferenceur": _get(record, "your_name") or _get(record,"referee_name"),
        "nom": _get(record, "referee_name"),
        "telephone": _get(record, "referee_phone", ""),
        "courriel": _get(record, "referee_email", ""),
        "autresInfos":_get(record,"additional_info",""),
        "payable": PAYABLE == True,
        **type_specific_data, # deplie le dictionnaire (result) dans le payload
    }
    return payload

In [0]:
df = spark.sql('''select * from bridge_dev.bridge_connect.referrals''' )
df.display()

# ---------------------TESTING MAIN_METHOD()---------------------

In [0]:
#COMPANY_EMAIL="general@bridgeconnect.azimut"
DELTA_TABLE = "adw_dev.bridge_connect.referrals"
# API_URL      = "https://dev-bridge-backend-app.azurewebsites.net/api"
# API_ENDPOINT = API_URL + "/Reference"
# API_ENDPOINT_KEY = API_URL + "/ConnexionExterne"
PAYABLE = True
# API_BEARER   = "vMysTHMhNEjlR9u1ySj0vmHoRBKznGN1NRGAoLuhXfmpE3AwZh"
TIMEOUT_SECS = 20

# -----------------------------Main()--------------------------

In [0]:
import json
import logging
import requests
from pyspark.sql.functions import col, lit
import psycopg2
from collections import defaultdict
import time


logging.basicConfig(  # configuration pour les logs
    level=logging.INFO, 
    format="[%(levelname)s] %(message)s"
)

API_CONF = {
    # "db5": {"API_URL": "https://references.ellipse-interne.ca/api", "API_BEARER": "JKSa5UaAX9WOMDGhGKow0heJnkwgoqwl76cU47XQPt7uGEuh7e", "API_ENDPOINT": "https://references.ellipse-interne.ca/api/Reference"},
    "db0": {"API_URL": "https://dev-bridge-backend-app.azurewebsites.net/api", "API_BEARER": "vMysTHMhNEjlR9u1ySj0vmHoRBKznGN1NRGAoLuhXfmpE3AwZh", "API_ENDPOINT": "https://dev-bridge-backend-app.azurewebsites.net/api/Reference"},
    "db1": {"API_URL": "https://dev-bridge-backend-app.azurewebsites.net/api", "API_BEARER": "vMysTHMhNEjlR9u1ySj0vmHoRBKznGN1NRGAoLuhXfmpE3AwZh", "API_ENDPOINT": "https://dev-bridge-backend-app.azurewebsites.net/api/Reference"},
    "db2": {"API_URL": "https://dev-bridge-backend-app.azurewebsites.net/api", "API_BEARER": "vMysTHMhNEjlR9u1ySj0vmHoRBKznGN1NRGAoLuhXfmpE3AwZh", "API_ENDPOINT": "https://dev-bridge-backend-app.azurewebsites.net/api/Reference"},
}
_last_activation = {}

def activate_api_token(source_db):
    cfg = API_CONF.get(source_db)
    api_url = cfg.get("API_URL")
    api_bearer = cfg.get("API_BEARER")
    src = source_db
    if not api_url or not api_bearer:
        logging.warning("API_URL or API_BEARER_TOKEN not set (source=%s)", source_db or "default")
        return

    # 20 minutes avant expiration activation
    now = time.time()
    last = _last_activation.get(src, 0)
    if now - last < 1200:  # 20 min * 60
        return  # valide

    activation_url = f"{api_url.rstrip('/')}/ConnexionExterne/{api_bearer}"
    try:
        r = requests.get(activation_url, timeout=TIMEOUT_SECS)
        logging.info("Activation[%s]: %s %s", src, r.status_code, r.reason)
        _last_activation[src] = now
    except Exception as e:
        logging.warning("Activation error [%s]: %s", src, e)


def send_referral(payload: dict, source_db): # Envoi de reference
    conf = API_CONF.get(source_db)
    url = conf["API_ENDPOINT"]
    api_bearer = conf["API_BEARER"]
    activate_api_token(source_db)

    if not conf:
        raise RuntimeError("API_CONF not set for this source_db : {source_db}")
    headers = {
    "Content-Type": "application/json",
    "X-api-key": api_bearer}

    r = requests.post(url, json=payload, headers=headers, timeout=20)
    
    return r

def mark_sent_in_delta( record_id): # Mettre a jour la table (colonne 'sent' = true)
    spark.sql(f"""
        UPDATE bridge_dev.bridge_connect.referrals
        SET sent = true
        WHERE id = '{record_id}'
    """)


def main():
    logging.info("Début d'envoi de references...")

    
    df = spark.sql('''select * from bridge_dev.bridge_connect.referrals where sent = false or sent is null  ''' )

    if df is None or len(df.columns) == 0:
        logging.warning("table '%s' not found or empty.", DELTA_TABLE)
        return {"ok": True, "updated": 0}

    count_pending = df.count()
    logging.info("Pending rows: %s", count_pending)
    if count_pending == 0:
        return {"ok": True, "updated": 0}


    updated = []
    errors = []

    
    for row in df.collect():
        record = row.asDict(recursive=True)
        try:
            payload = transform_to_api_format(record)

            resp = send_referral(payload,row["source_db"])
            time.sleep(11)
            if getattr(resp, "status_code", None) == 401:
                activate_api_token(row["source_db"])
                resp = send_referral(payload, row["source_db"])
                time.sleep(11)
            ok = (getattr(resp, "status_code", None) == 200)  
            body = {}
            try:
                body = resp.json()
                print(body)
            except Exception:
                pass

            if ok and body.get("id"): 
                mark_sent_in_delta(payload["id"])
                updated.append((payload["id"],row["source_db"]))
                logging.info("Marked sent in Delta: %s", payload["id"])
            else:
                logging.warning("Referral not accepted for id=%s; status=%s body=%s",
                                payload.get("id"), getattr(resp, "status_code", None), getattr(resp, "text", ""))

        except Exception as e:
            logging.exception("Error processing id=%s", record.get("id"))
            errors.append({"id": record.get("id"), "error": str(e)})

    
    logging.info("Updated count: %d", len(updated))
    if len(updated) > 0 :
        display(spark.createDataFrame(updated)) 
    if errors:
        logging.warning("Errors count: %d", len(errors))
    else : pass

    #----------------delete from dbs ---------
    # ids_by_src = defaultdict(list)
    # for _id, src in updated:
    #     ids_by_src[src].append(_id)

    # sources = {
    #     "db0": {"host":"ep-misty-field-aeg07ur8.c-2.us-east-2.aws.neon.tech", "port":"5432", "db":"neondb", "user":"neondb_owner", "pwd":"npg_ZRuSiM2IBn7p", "sslmode":"require", "schema":"public", "table":"referrals"},

    #     "db1": {"host":"ep-shy-rice-adbb0t2f.c-2.us-east-1.aws.neon.tech", "port":"5432", "db":"neondb", "user":"neondb_owner", "pwd":"npg_JAwf5danYy8q", "sslmode":"require", "schema":"public", "table":"referrals"},

    #     "db2": {"host":"ep-bitter-cake-adhdrtjy.c-2.us-east-1.aws.neon.tech", "port":"5432", "db":"neondb", "user":"neondb_owner", "pwd":"npg_meDT9zHBi6hl", "sslmode":"require", "schema":"public", "table":"referrals"},
    # }

    # for name, con in sources.items():
    #     ids = ids_by_src.get(name, [])
        
        
    #     if not ids:
    #         print(f"{name}: nothing to delete.")
    #         continue

    #     sql = f''' DELETE FROM "{con["schema"]}"."{con["table"]}" WHERE id = ANY(%s); '''

    #     with psycopg2.connect(
    #         host=con["host"], port=con["port"], dbname=con["db"],
    #         user=con["user"], password=con["pwd"], sslmode=con["sslmode"]
    #     ) as conn:
    #         with conn.cursor() as cur:
    #             cur.execute(sql, (ids,))
    #             print(f"{name}: deleted {cur.rowcount} row(s)")
    #-----------------------------------------------------------------------------------------

    return {"ok": len(errors) == 0, "updated": len(updated), "errors": errors}



result = main()
print(result)
